# Homework 5: Coding

**Due Monday October 11th, 11:59pm.**


**In order to avoid module version issues, please complete this assignment on Colab.**

**Submit hw5.py file to Gradescope (note there is no autograder for this assignment).**

In [ ]:
"""
Import libraries that you might require
"""

import numpy as np
import math
import matplotlib.pyplot as plt
import operator
from sklearn.metrics import accuracy_score
import sklearn.model_selection as ms


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier


In [ ]:
"""
Load data (MNIST digits dataset).

Note that we will skip the validation phase for
this exercise as by now you are pretty familiar with the typical Machine Learning
pipeline.
"""

from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape)

X = digits['data']
y = digits['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:1500], y[:1500]
X_test, y_test = X[1500:], y[1500:]


(1797, 64)


# Question 2: Performance Comparisons for three ML algorithms

## 2.0 Accuracy

In [ ]:
def train(models, X_train, y_train, X_test, y_test):
  """
  Trains several models and returns the test accuracy for each of them
  Args:
      models: list of model objects
  Returns:
      score (float): list of accuracies of the different fitted models on test set
  """

  # To complete: train and test each model in a for lop

  accuracies = []


  return accuracies

## 2.1 Random Forest Classifier

In [ ]:
def modelRF(n_estimators):
  """
  Creates model objects for the Random Forest Classifier.
  See the documentation in sklearn here:
  https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
  Args:
    n_estimators: list of hyper parameters
  return:
    list of classifiers
  """
  
  list_n_estimators = n_estimators
  random_state = 20 # Do not change this random_state


  objs_RFC = []
  
  # To complete: Create a list of objects for the classifier for each of the above "n_estimators"

  return objs_RFC
 

# To complete: call the above function to train and test the Random Forest Classifier
    

## 2.2 Kernel SVM

In [ ]:
def modelKSVM():
  """
  Creates model objects for the Kernel SVM.
  See the documentation in sklearn here:
  https://scikit-learn.org/stable/modules/svm.html
  """
  
  list_kernel_type = ['linear', 'poly', 'rbf']
  random_state = 20 # Do not change this random_state

  objs_KSVM = []
  
  # To complete: Create a list of objects for the classifier for each of the above "kernel" types

  return objs_KSVM

# To complete: Call the above function to train and test the Kernel SVM
    

## 2.3 Multi Layer Perceptron

In [ ]:
def modelMLP():
  """
  Creates model objects for the Multi Layered Perceptron.
  See the documentation in sklearn here:
  https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
  """
  
  
  layerSizes = [(3), (10), (10,10,10), (20,50,20)]
  random_state = 20 # Do not change this random_state
  max_iter = 2000 # fixed max_iter
  
  objs_MLP = []

  # To complete: Create a list of objects for the classifier for each of the above "layerSizes"

  return objs_MLP

# To complete: Call the above function to train and test the Multi Layer Perceptron
    

## 2.4 AdaBoost

In [ ]:
def modelAdaBoost():
  """
  Creates model objects for the AdaBoost.
  See the documentation in sklearn here:
  https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
  """
  num_estimators = [1,5,10,50,100,150]
  learning_rate = 0.1
  max_depth = 3
  random_state = 20 # Do not change this random_state
  # To complete: Create a list of objects for the classifier for each of combination of above num_estimators and learning_rate
  obj_boost = []

  return obj_boost

# To complete: Call the above function to train and test the AdaBoost Classifier


In [ ]:
!rm -r -f ./logs

LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
# Code referenced from https://gist.github.com/gyglim/1f8dfb1b5c82627ae3efcfbbadb9f514
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.create_file_writer(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        with self.writer.as_default():
            tf.summary.scalar(name=tag, data=value, step=step)
        self.writer.flush()

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

            # Create an Image object as a Summary value
            with self.writer.as_default():
                tf.summary.image(name='%s/%d' % (tag, i), data=s.getvalue(), step=step)

        # Create and write Summary
        self.writer.flush()
        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

        # Drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        with self.writer.as_default():
            tf.summary.histogram(name=tag, data=hist, step=step)
        self.writer.flush()
logger = Logger('./logs')

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms

In [ ]:
# Define a transform to normalize the data

#TODO : Set the value of mean and the standard deviation to 
#       normalize the image from range [0,1] to the range [-1, 1]


#Begin Your Code

mean = 
std = 

#End Your Code

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((mean,), (std,))
                                ])


#TODO : Select suitable value of batch_sizes.

#Begin Your Code

train_batch_size = 
test_batch_size = 

#End Your Code

# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)


# Classes
classes = {       0 :'T-shirt/top',
                  1 :'Trouser',
                  2 :'Pullover',
                  3 :'Dress',
                  4 :'Coat',
                  5 :'Sandal',
                  6 :'Shirt',
                  7 :'Sneaker',
                  8 :'Bag',
                  9 :'Ankle boot'}

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

# Functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    
    figure(num=None, figsize=(8, 6), dpi=150, edgecolor='k')
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #TODO : Design your network, you are allowed to explore your own architecture
        #       But you should achieve a better overall accuracy than the baseline network.
        #       Also, if you do design your own network, include an explanation 
        #       for your choice of network and how it may be better than the 
        #       baseline network in your latex.
        
        #Begin Your Code


        #End Your Code

    def forward(self, x):

      #TODO : Implement the forward function that applies the layers you have created to the input

      #Begin Your Code


      #End Your Code


net = Net()

In [ ]:
import torch.optim as optim

#TODO : Use appropriate loss criterion and optimizer 

#Begin Your Code

criterion = 
optimizer = 

#End Your Code

In [ ]:
overall_step = 0

#TODO : Select appropriate number of epochs

#Begin Your Code

epochs =

#End Your Code


for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        #TODO : Make predictions, calculate accuracy and update your weights once

        #Begin Your Code

        # zero the parameter gradients

        # forward + backward + optimize

        #End Your Code

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('Epoch: %d, Batch: %5d, loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
            #Any thing that is added to the "info" gets plotted in tensorboard
            #TODO : Add the plots in Tensorboard to the report and explain what is happening
            info = {'loss' : loss.item(), 'accuracy': accuracy.item()}
            for tag, value in info.items():
                logger.scalar_summary(tag, value, overall_step+1)

print('Finished Training')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

#TODO : Report this accuracy in your report.

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


# Turning it in

**This notebook will not be autograded, so no need to comment out code outside of functions.**

1. Download this notebook as a `hw5.py` file with the functions implemented and the sandbox code commented out
  - go to "File -> Download .py"
  
2. Submit `hw4.py` file to Gradescope (you can do this as many times as you'd like before the deadline)